# Audit data entry



In [1]:
from __future__ import print_function

from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown, Layout, Box
import ipywidgets as widgets
from IPython.display import display, HTML

from collections import OrderedDict
from itertools import product
import math
import json

import numpy as np
from ballot_comparison import ballot_comparison_pvalue
from fishers_combination import  maximize_fisher_combined_pvalue, create_modulus
from sprt import ballot_polling_sprt

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

from suite_tools import write_audit_parameters, write_audit_results, \
        check_valid_audit_parameters, check_valid_vote_counts, \
        check_overvote_rates, find_winners_losers, print_reported_votes, \
        estimate_n, estimate_escalation_n, \
        parse_manifest, unique_manifest, find_ballot, \
        audit_contest
        

# Input the global audit parameters.

For an audit, you should input the following global parameters in the cell below:

* contest-specific parameters:
    * `risk_limit`: the risk limit for the audit
    * `stratum_sizes`: total ballots in the two strata, [CVR total, no-CVR total]
    * `num_winners`: number of winners in the contest
* software parameters:
    * `seed`: the numeric seed for the pseudo-random number generator used to draw samples of ballots. Use, e.g., 20 rolls of a 10-sided die 
    * `gamma`: the gamma parameter used in the ballot-polling method from Lindeman and Stark (2012). Default value of 1.03905 is generally accepted
    * `lambda_step`: the initial step size in the grid search over the way error is allocated across the CVR and no-CVR strata in SUITE. Default 0.05 is acceptable
* initial sample size estimate parameters:
    * `o1_rate`: expected rate of 1-vote overstatements in the CVR stratum
    * `o2_rate`: expected rate of 2-vote overstatements in the CVR stratum
    * `u1_rate`: expected rate of 1-vote understatements in the CVR stratum
    * `u2_rate`: expected rate of 2-vote understatements in the CVR stratum
    * `n_ratio`: what fraction of the sample is taken from the CVR stratum. Default is to allocate sample in proportion to ballots cast in each stratum.
    


In [2]:
# global audit parameters

# contest-specific parameters
risk_limit = 0.05    # risk limit
stratum_sizes = [100000, 5000]  # total ballots in the two strata, CVR, no-CVR
num_winners = 2       # maximum number of winners, per social choice function


# software parameters
seed = 12345678901234567890  # use, e.g., 20 rolls of a 10-sided die
gamma=1.03905         # gamma from Lindeman and Stark (2012)
lambda_step = 0.05    # stepsize for the discrete bounds on Fisher's combining function

# initial sample size parameters
o1_rate = 0.002       # expect 2 1-vote overstatements per 1000 ballots in the CVR stratum
o2_rate = 0           # expect 0 2-vote overstatements
u1_rate = 0           # expect 0 1-vote understatements
u2_rate = 0           # expect 0 2-vote understatements
n_ratio = stratum_sizes[0]/np.sum(stratum_sizes) 
                     # allocate sample in proportion to ballots cast in each stratum

In [3]:
check_valid_audit_parameters(risk_limit, lambda_step, o1_rate, o2_rate, \
                                 u1_rate, u2_rate, stratum_sizes, n_ratio, num_winners)

In [4]:
write_audit_parameters("audit_parameters.json",\
                       risk_limit, stratum_sizes, num_winners, seed, gamma, \
                       lambda_step, o1_rate, o2_rate, \
                       u1_rate, u2_rate, n_ratio)

# Enter the reported votes

Candidates are stored in a data structure called a dictionary. Enter the candidate name and the votes in each stratum, [votes in CVR stratum, votes in no-CVR stratum], in the cell below. The following cell will calculate the vote totals, margins, winners, and losers.

In [5]:
# input number of winners
# input names as well as reported votes in each stratum

# candidates are a dict with name, [votes in CVR stratum, votes in no-CVR stratum]
candidates = { "candidate 3": [30000, 500],
               "candidate 2": [50000, 1000],
               "candidate 1": [10000, 500],
               "candidate 4": [500, 10]}

# Run validity check on the input vote totals
check_valid_vote_counts(candidates, stratum_sizes)

In [16]:
sampled_ballots = 10

candidate_list = list(sorted(candidates))
candidate_list.append('non-vote')
w = []

for i in range(sampled_ballots):
    w.append(
        widgets.Dropdown(options=candidate_list, description='hand-to-eye:', disabled=False))
    display('ballot' + str(i) + w[i])

TypeError: must be str, not Dropdown

In [20]:
# step 1: expand the ballot manifest into a dict. keys are batches, values are ballot numbers.
cvr_manifest_parsed = parse_manifest(ballot_manifest_cvr)
poll_manifest_parsed = parse_manifest(ballot_manifest_poll)

In [21]:
# count ballots listed in the manifests
listed_cvr = np.sum([len(v) for v in cvr_manifest_parsed.values()])
listed_poll = np.sum([len(v) for v in poll_manifest_parsed.values()])

# test that manifest matches reported ballot totals

assert listed_cvr == stratum_sizes[0]
assert listed_poll == stratum_sizes[1]

In [22]:
# step 2: give ballots unique IDs

unique_cvr_manifest = unique_manifest(cvr_manifest_parsed)
unique_poll_manifest = unique_manifest(poll_manifest_parsed)

**KELLIE TO DOS:** 
* for the CVR stratum, include ballot multiplicity
* make ballot manifest syntax consistent with the spec

In [23]:
# step 3: look up sample values

print("CVR Stratum")

cvr_sample = []
for s in sample1:
    original_ballot_label, batch_label, which_ballot = find_ballot(s, \
                                                                   unique_cvr_manifest, \
                                                                   cvr_manifest_parsed)
    cvr_sample.append([s, batch_label, which_ballot])

cvr_sample.sort(key=lambda x: x[2]) # Sort second on order within batches
cvr_sample.sort(key=lambda x: x[1]) # Sort first based on batch label
cvr_sample.insert(0,["sampled ballot", "batch label", "which ballot in batch"])

display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in cvr_sample)
        )
 ))

CVR Stratum


sampled ballot,batch label,which ballot in batch
2081,1,2081
3974,1,3974
8262,1,8262
9973,1,9973
10908,2,908
12244,2,2244
16602,2,6602
17757,2,7757
20823,2,10823
20934,2,10934


In [24]:
print("Polling Stratum")

nocvr_sample = []
for s in sample2:
    original_ballot_label, batch_label, which_ballot = find_ballot(s, \
                                                                   unique_poll_manifest, \
                                                                   poll_manifest_parsed)
    nocvr_sample.append([s, batch_label, which_ballot])

nocvr_sample.sort(key=lambda x: x[2]) # Sort second on order within batches
nocvr_sample.sort(key=lambda x: x[1]) # Sort first based on batch label
nocvr_sample.insert(0,["sampled ballot", "batch label", "which ballot in batch"])

display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in nocvr_sample)
        )
 ))

Polling Stratum


sampled ballot,batch label,which ballot in batch
1133,2,133
4784,2,3784


# Enter the sample data

Sample statistics for the CVR stratum (stratum 1)

In [25]:
# CVR stratum sample size
n1 = 60

In [26]:
# Number of observed...

def cvr_audit_inputs(o1, o2, u1, u2):
    return (o1, o2, u1, u2)

cvr_stats = interactive(cvr_audit_inputs,
                             o1 = widgets.IntSlider(min=0,max=n1,value=0),
                             u1 = widgets.IntSlider(min=0,max=n1,value=0),
                             o2 = widgets.IntSlider(min=0,max=n1,value=0),
                             u2 = widgets.IntSlider(min=0,max=n1,value=0))
display(cvr_stats)

(5, 0, 0, 0)

In [27]:
(o1, o2, u1, u2) = [cvr_stats.children[i].value for i in range(4)]

Sample statistics for the no-CVR stratum (stratum 2)

In [28]:
# Number of votes for each candidate
# recall that in the provided example, n2=3 so the totals here must add up to <= 3.
n2 = 3

In [29]:
nocvr_widgets=[]

# create the widgets
for name in candidates.keys():
    nocvr_widgets.append(widgets.IntSlider(value=0,min=0,max=n2,description=name))

# It'd be great to constrain their sum to be <= n2

#for widget in nocvr_widgets:
#    widget.observe(lambda change:myfct(change,nocvr_widgets),names='value',type='change')

# group the widgets into a FlexBox
nocvr_audit_inputs = widgets.VBox(children=nocvr_widgets)

# display the widgets
display(nocvr_audit_inputs)

In [30]:
# no-CVR sample is stored in a dict with name, votes in the sample

observed_poll = {}
for widget in nocvr_widgets:
    observed_poll[widget.description] = widget.value

assert np.sum(list(observed_poll.values())) <= n2, "Too many ballots input"
print(observed_poll)

{'candidate 4': 0, 'candidate 3': 1, 'candidate 1': 0, 'candidate 2': 2}


# What's the risk for this sample?

The audit looks at every (winner, loser) pair in each contest. Auditing continues until there is strong evidence that every winner in a contest got more votes than every loser in the contest. It does this by considering (winner, loser) pairs. The SUITE risk for every pair will appear beneath the cell below after it is run. The audit continues until all the numbers are not larger than the risk limit. E.g., if the risk limit is 10%, the audit stops when the numbers in the table are all less than 0.1.

In [31]:
# Find audit p-values across (winner, loser) pairs

audit_pvalues = audit_contest(candidates, winners, losers, stratum_sizes, \
                  n1, n2, o1, o2, u1, u2, observed_poll, \
                  risk_limit=risk_limit, gamma=gamma, stepsize=lambda_step)
audit_pvalues

{('candidate 2', 'candidate 1'): 0.0008703740183476638,
 ('candidate 2', 'candidate 4'): 3.310313991689018e-05,
 ('candidate 3', 'candidate 1'): 0.23272911575919975,
 ('candidate 3', 'candidate 4'): 0.01806178712685269}

In [32]:
# Track contests not yet confirmed

contests_not_yet_confirmed = [i[0] for i in audit_pvalues.items() \
                              if i[1]>risk_limit]
print("Pairs not yet confirmed:\n", contests_not_yet_confirmed)

Pairs not yet confirmed:
 [('candidate 3', 'candidate 1')]


In [33]:
# Save everything to file


write_audit_results("audit_results.json", \
                        n1, n2, sample1, sample2, \
                        o1, o2, u1, u2, observed_poll, \
                        audit_pvalues)

# Escalation guidance: how many more ballots should be drawn?

This tool estimates how many more ballots should be examined to confirm any remaining contests. The enlarged sample size is based on the following:
* ballots that have already been sampled
* assumption that we will continue to see overstatements and understatements at the same rate that they've been observed in the sample so far
* assumption that vote proportions in the ballot-polling stratum will reflect the reported proportions

Given these additional numbers, return to the sampling tool and draw additional ballots, find them with the ballot manifest tool, update the observed sample values, and rerun the SUITE risk calculations.

In [34]:
sample_sizes_new = {}

# Add a reminder note about the candidate dict structure.

for k in contests_not_yet_confirmed:
    sample_sizes_new[k] = estimate_escalation_n(\
                                 N_w1 = candidates[k[0]][0],\
                                 N_w2 = candidates[k[0]][1],\
                                 N_l1 = candidates[k[1]][0],\
                                 N_l2 = candidates[k[1]][1],\
                                 N1 = stratum_sizes[0],\
                                 N2 = stratum_sizes[1],\
                                 n1 = n1,\
                                 n2 = n2,\
                                 o1_obs = o1,\
                                 o2_obs = o2,\
                                 u1_obs = u1,\
                                 u2_obs = u2,\
                                 n2l_obs = observed_poll[k[1]],\
                                 n2w_obs = observed_poll[k[0]],\
                                 n_ratio = n_ratio,\
                                 risk_limit = risk_limit,\
                                 gamma = gamma,\
                                 stepsize = lambda_step)

In [35]:
sample_size_new = np.amax([v[0]+v[1] for v in sample_sizes_new.values()])
n1_new = np.amax([v[0] for v in sample_sizes_new.values()])
n2_new = np.amax([v[1] for v in sample_sizes_new.values()])

print(sample_sizes_new, '\n\nExpected minimum sample size:', sample_size_new)
print("\nBallots to draw in the CVR stratum:", n1_new - n1)
print("Ballots to draw in the no-CVR stratum:", n2_new - n2)

{('candidate 3', 'candidate 1'): (113, 5)} 

Expected minimum sample size: 118

Ballots to draw in the CVR stratum: 53
Ballots to draw in the no-CVR stratum: 2
